# Ploting the polygon after reterving from Postgres DB

<font size="4">Sometimes working with **raster** data could be very defficult especially when we are working on large area. In this case, the best solution would be to *convert raster to polygon* and save it in the DB. After that, it is required to *import the polygons into the python environment* in **Dataframe** format and convert it into the **Geodataframe**.</font>

In [1]:
import pandas as pd
import geopandas as gpd
import rasterio as rs
import contextily as ctx
import psycopg2
import folium
from folium import plugins
from getpass import getpass
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
import os
import descartes
import numpy as np
from shapely import wkt
from tqdm import tqdm

In [2]:
password = getpass()

 ············


In [3]:
conn = psycopg2.connect("dbname='***' user='***' host='***' password='%s'" % (password))

In [4]:
sql="""select *, ST_AsText(geom) as geometry  from soilmoisture.uerra_harmonie_nordics_sm_l1_ymonmax_vector;"""

In [5]:
df=pd.read_sql_query(sql,conn)
df = df.drop(['geom'], axis=1)

<font size="4" color="orange"> We use `wshapely.wkt` sub-module to parse wkt format:</font>

In [6]:
df['geometry'] = df['geometry'].apply(wkt.loads)

In [7]:
df.head().T

,0,1,2,3,4
fid,1,2,3,4,5
ymonmax_january,0.34861,0.323368,0.250231,0.254337,0.288951
ymonmax_february,0.288586,0.327911,0.205932,0.202562,0.237086
ymonmax_march,0.168546,0.309034,0.107765,0.0946213,0.150582
ymonmax_april,0.280642,0.287013,0.237945,0.225098,0.260775
ymonmax_may,0.292876,0.313154,0.288522,0.289397,0.290108
ymonmax_june,0.286224,0.288791,0.285195,0.290401,0.293585
ymonmax_july,0.327459,0.279482,0.327792,0.337413,0.341126
ymonmax_august,0.310177,0.306477,0.30817,0.317077,0.316248
ymonmax_september,0.302313,0.282393,0.308092,0.309723,0.323115


<font size="4" color="orange"> Converting **Dataframe** into **Geodataframe**:</font>

In [9]:
gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf.crs = {'init':'epsg:3035'}

In [10]:
print(gdf.crs)

+init=epsg:3035 +type=crs


In [ ]:
clm_name = gdf.columns
fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(10,12))
for i in tqdm(range(0, 4)):
    for j in range(0, 3):
        clm = ((i * 3) + j+1)
        gdf.plot(ax=axes[i,j], column = clm_name[clm], cmap = 'RdYlGn_r')
        axes[i,j].set_axis_off()
        #ctx.add_basemap(axes[i,j])
        axes[i,j].set_title(clm_name[clm])
        
plt.savefig('max_soilmoisture.png',dpi=500, transparent=True)

100%|██████████| 4/4 [01:05<00:00, 16.39s/it]
